In [ ]:
# !pip install autocorrect
# !pip install textblob
# !pip install symspellpy
# !pip install pyaspell
# !pip install fuzzywuzzy
# !pip install hunspell
#!pip install better_profanity

In [2]:
import pandas as pd

In [3]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import emoji
from nltk.corpus import stopwords, wordnet
import time
from pathlib import Path

from collections import Counter
from spellchecker import SpellChecker
import os

from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer, PorterStemmer
from nltk.stem import WordNetLemmatizer
from ftfy import fix_encoding
import spacy

#nlp = spacy.load("en_core_web_sm")

pd.set_option('display.max_colwidth', None)

In [12]:

current_dir = os.getcwd()
csv_path  = os.path.abspath(os.path.join(current_dir, '../../../data/twitter_hate-speech/train_cleaned.csv'))
dict_path = os.path.abspath(os.path.join(current_dir, '../nasiba/data/frequency_dictionary_en_82_765.txt'))
df_cleaned = pd.read_csv(csv_path, encoding='utf-8')

df_cleaned.head(1)



,id,label,tweet,tweet_cleaned,user_handle,hashtags,emojis
0,1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,father selfish drag kid run,1,['#run'],NaN


#### Spell Correction:

In [162]:
import re
from symspellpy import SymSpell, Verbosity

def apply_symspell_to_dataframe(df, column_name,path):
    # SymSpell initialisieren
    sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
    sym_spell.load_dictionary(path, term_index=0, count_index=1)

    # Funktion zur Rechtschreibprüfung für einen Text
    def correct_spelling(text):
        if not isinstance(text, str):
            return {"misspelled": [], "corrected": []}  # Für nicht-String-Eingaben (z. B. NaN)
        
        words = text.split()
        misspelled = []
        corrected = []
        
        for word in words:
            lookup_result = sym_spell.lookup(word.lower(), Verbosity.CLOSEST, max_edit_distance=2)
            if lookup_result:
                suggestion = lookup_result[0].term
                if suggestion.lower() != word.lower():  # Wenn das Wort korrigiert wurde
                    misspelled.append(word)
                    corrected.append(suggestion)
        
        return {"misspelled": misspelled, "corrected": corrected}

    # Anwenden der Rechtschreibprüfung auf die Spalte
    spellcheck_results = df[column_name].apply(correct_spelling)
    
    # Hinzufügen der Ergebnisse zu neuen Spalten
    df["misspelled_words"] = spellcheck_results.apply(lambda x: x["misspelled"])
    df["corrected_words"] = spellcheck_results.apply(lambda x: x["corrected"])

    return df

# Anwendung der Funktion
df_cleaned = apply_symspell_to_dataframe(df_cleaned, "tweet_cleaned",dict_path)

# Filtern der Zeilen mit falsch geschriebenen Wörtern
df_filtered = df_cleaned[df_cleaned["misspelled_words"].apply(len) > 0]

# Anzeigen der gewünschten Spalten
df_filtered[['misspelled_words', 'corrected_words', 'tweet', 'tweet_cleaned','label']]


,misspelled_words,corrected_words,tweet,tweet_cleaned,label
1,[lyft],[left],@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thank lyft credit use cause offer van,0
2,[bihday],[birthday],bihday your majesty,bihday majesty,0
8,"[allin, cavs]","[allen, cars]",we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers …,win land allin cavs champion cleveland,0
10,"[(, mom, ), forex]","[a, mon, a, fore]",↝ #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may #blog #silver #gold #forex,ireland consumer price index ( mom ) climb previous may blog silver gold forex,0
13,"[cnn, tcot]","[can, tot]",@user #cnn calls #michigan middle school 'build the wall' chant '' #tcot,cnn call middle school build wall chant tcot,1
...,...,...,...,...,...
27638,[hashtag],[hashing],@user @user you don't have the balls to hashtag me as a but you say i am to weasel away.. lumpy tony.. dipshit.,ball hashtag say away lumpy tony,1
27639,[oh],[of],"makes you ask yourself, who am i? then am i anybody? until ....god . oh thank you god!",make ask anybody god oh thank god,1
27643,[newyork],[network],good morning #instagram #shower #water #berlin #berlincitygirl #girl #newyork #zürich #genf #bern,good morning instagram shower water berlin girl newyork,0
27647,[youuu],[you],ate @user isz that youuu?😍😍😍😍😍😍😍😍😍❤️,eat youuu red,0


In [163]:
df_filtered[df_filtered['label']==1]


,id,label,tweet,tweet_cleaned,user_handle,hashtags,emojis,misspelled_words,corrected_words
13,14,1,@user #cnn calls #michigan middle school 'build the wall' chant '' #tcot,cnn call middle school build wall chant tcot,1,"['#cnn', '#michigan', '#tcot']",NaN,"[cnn, tcot]","[can, tot]"
14,15,1,no comment! in #australia #opkillingbay #seashepherd #helpcovedolphins #thecove #helpcovedolphins,comment australia opkillingbay seashepherd thecove,0,"['#australia', '#opkillingbay', '#seashepherd', '#helpcovedolphins', '#thecove', '#helpcovedolphins']",NaN,[thecove],[they've]
17,18,1,retweet if you agree!,retweet agree,0,[],NaN,[retweet],[between]
81,83,1,how the #altright uses &amp; insecurity to lure men into #whitesupremacy,altright use amp man whitesupremacy,0,"['#altright', '#whitesupremacy']",NaN,[altright],[alright]
112,115,1,@user why not @user mocked obama for being black. @user @user @user @user #brexit,mock obama black brexit,6,['#brexit'],NaN,[brexit],[credit]
...,...,...,...,...,...,...,...,...,...
27626,31934,1,@user judd is a &amp; #homophobic #freemilo #milo #freemilo #milo #freemilo #milo #freemilo #milo #freemilo,amp homophobic freemilo milo freemilo milo freemilo milo freemilo milo freemilo,1,"['#homophobic', '#freemilo', '#milo', '#freemilo', '#milo', '#freemilo', '#milo', '#freemilo', '#milo', '#freemilo']",NaN,"[freemilo, freemilo, freemilo, freemilo, freemilo]","[freewill, freewill, freewill, freewill, freewill]"
27627,31935,1,lady banned from kentucky mall. @user #jcpenny #kentucky,lady ban kentucky mall jcpenny kentucky,1,"['#jcpenny', '#kentucky']",NaN,[jcpenny],[penny]
27637,31947,1,@user omfg i'm offended! i'm a mailbox and i'm proud! #mailboxpride #liberalisme,omfg offend proud,1,"['#mailboxpride', '#liberalisme']",NaN,[omfg],[mfg]
27638,31948,1,@user @user you don't have the balls to hashtag me as a but you say i am to weasel away.. lumpy tony.. dipshit.,ball hashtag say away lumpy tony,2,[],NaN,[hashtag],[hashing]


In [148]:
# spell= SpellChecker()
# spell_aut = Speller(lang='en')
# misspelled = spell.unknown('retweet')
# print(misspelled)  # Output: {'wrld'}

# suggestions = spell.candidates('retweet')
# print(suggestions)  # Output: {'world'}

# # correction_spell = spell.correction('I havv a guud ideea')
# # print(correction_spell)  # Output: 'world'


# suggestions_unk = spell_aut.autocorrect_sentence('retweet')
# print(suggestions_unk  )  # Output: 'world'

# suggestions_aut = spell_aut.get_candidates('retweet')
# print(suggestions_aut )  # Output: 'world'

# # correction_aut = spell_aut('I havv a guud ideea')
# # print(correction_aut )  # Output: 'world'

# text = TextBlob('retweet')
# print(text.correct())  # "I have a spelling error in this sentence."

sym_spell = SymSpell()
sym_spell.load_dictionary(dict_path, 0, 1)
words = ['retweet','bihday']
suggestions = sym_spell.lookup('retweet', Verbosity.CLOSEST, max_edit_distance=2)
for suggestion in suggestions:
    print(suggestion.term)  # "spelling"


print(process.extract("speling", words))  # [('spelling', 90), ('spilling', 80)]


between
retreat
retest
tweet
[('bihday', 15), ('retweet', 14)]


In [75]:

spell = SpellChecker()
def identify_misspelled_words(text):
    # Sicherstellen, dass text ein String ist
    if not isinstance(text, str):
        return []  # Leere Liste zurückgeben, wenn der Wert kein String ist

    words = text.split()

    misspelled = [
        word for word in words 
        if not any(emoji.is_emoji(char) for char in word)  # Ignoriere Emojis
        and not word.startswith('@')  # Ignoriere Wörter, die mit @ beginnen
        and not word.startswith('#')  # Ignoriere Wörter, die mit # beginnen
        and word in spell.unknown([word])  # Führe Rechtschreibprüfung auf verbleibende Wörter durch
        and not word.endswith('!')
        and not word.endswith(']')
        and not word.startswith('[')
        and not word.endswith('.')
    ]
    return misspelled


df_cleaned['misspelled_words'] = df_cleaned['tweet_cleaned'].apply(identify_misspelled_words)

# Zeilen zählen, bei denen die Spalte 'misspelled_words' nicht leer ist
misspelled_count = df_cleaned[df_cleaned['misspelled_words'].str.len() > 0].shape[0]
total_count = df_cleaned.shape[0]

print(misspelled_count)
print(total_count )
percentage = (misspelled_count / total_count) * 100
print(f"Prozentualer Anteil der Zeilen mit falsch geschriebenen Wörtern: {percentage:.2f}%")


15712
27650
Prozentualer Anteil der Zeilen mit falsch geschriebenen Wörtern: 56.82%


In [76]:
df_analyse_label_0= df_cleaned[df_cleaned['label']==0]
df_analyse_label_1= df_cleaned[df_cleaned['label']==1]

In [77]:

spell = SpellChecker()
def identify_misspelled_words(text):
    # Sicherstellen, dass text ein String ist
    if not isinstance(text, str):
        return []  # Leere Liste zurückgeben, wenn der Wert kein String ist

    words = text.split()

    misspelled = [
        word for word in words 
        if not any(emoji.is_emoji(char) for char in word)  # Ignoriere Emojis
        and not word.startswith('@')  # Ignoriere Wörter, die mit @ beginnen
        and not word.startswith('#')  # Ignoriere Wörter, die mit # beginnen
        and word in spell.unknown([word])  # Führe Rechtschreibprüfung auf verbleibende Wörter durch
        and not word.endswith('!')
        and not word.endswith(']')
        and not word.startswith('[')
        and not word.endswith('.')
    ]
    return misspelled


df_analyse_label_1['misspelled_words'] = df_analyse_label_1['tweet_cleaned'].apply(identify_misspelled_words)

# Zeilen zählen, bei denen die Spalte 'misspelled_words' nicht leer ist
misspelled_count = df_analyse_label_1[df_analyse_label_1['misspelled_words'].str.len() > 0].shape[0]
total_count = df_analyse_label_1.shape[0]

print(misspelled_count)
print(total_count )
percentage = (misspelled_count / total_count) * 100
print(f"Prozentualer Anteil der Zeilen mit falsch geschriebenen Wörtern: {percentage:.2f}%")


1106
1811
Prozentualer Anteil der Zeilen mit falsch geschriebenen Wörtern: 61.07%


C:\Users\Nasiba\AppData\Local\Temp\ipykernel_16000\2653595961.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analyse_label_1['misspelled_words'] = df_analyse_label_1['tweet_cleaned'].apply(identify_misspelled_words)


In [ ]:
import re
from symspellpy import SymSpell, Verbosity

def apply_symspell_to_dataframe(df, column_name,path):
    # SymSpell initialisieren
    sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
    sym_spell.load_dictionary(path, term_index=0, count_index=1)

    # Funktion zur Rechtschreibprüfung für einen Text
    def correct_spelling(text):
        if not isinstance(text, str):
            return {"misspelled": [], "corrected": []}  # Für nicht-String-Eingaben (z. B. NaN)
        
        words = text.split()
        misspelled = []
        corrected = []
        
        for word in words:
            lookup_result = sym_spell.lookup(word.lower(), Verbosity.CLOSEST, max_edit_distance=2)
            if lookup_result:
                suggestion = lookup_result[0].term
                if suggestion.lower() != word.lower():  # Wenn das Wort korrigiert wurde
                    misspelled.append(word)
                    corrected.append(suggestion)
        
        return {"misspelled": misspelled, "corrected": corrected}

    # Anwenden der Rechtschreibprüfung auf die Spalte
    spellcheck_results = df[column_name].apply(correct_spelling)
    
    # Hinzufügen der Ergebnisse zu neuen Spalten
    df["misspelled_words"] = spellcheck_results.apply(lambda x: x["misspelled"])
    df["corrected_words"] = spellcheck_results.apply(lambda x: x["corrected"])

    return df

# Anwendung der Funktion
df_cleaned = apply_symspell_to_dataframe(df_cleaned, "tweet_cleaned",dict_path)

# Filtern der Zeilen mit falsch geschriebenen Wörtern
df_filtered = df_cleaned[df_cleaned["misspelled_words"].apply(len) > 0]

# Anzeigen der gewünschten Spalten
df_filtered[['misspelled_words', 'corrected_words', 'tweet', 'tweet_cleaned','label']]


In [ ]:

from autocorrect import Speller

# Initialisiere den Autocorrect-Checker für Englisch
spell = Speller(lang='en')
def apply_spell_to_dataframe(df, column_name):
    def correct_spelling(text):
        if not isinstance(text, str):
            return {"misspelled": [], "corrected": []}  # Leeres Dictionary, wenn kein String

        words = text.split()

        misspelled = []
        corrected = []

        for word in words:
        # Überprüfen, ob das Wort korrigiert werden muss
            corrected_word = spell(word)
            if corrected_word != word:  # Nur korrigierte Wörter berücksichtigen
                misspelled.append(word)
                corrected.append(corrected_word)

    # Rückgabe von None, wenn keine Fehler gefunden werden
        if len(misspelled) == 0:
            return None
        else:
            return {"misspelled": misspelled, "corrected": corrected}

    spellcheck_results = df[column_name].apply(correct_spelling)
    
    # Hinzufügen der Ergebnisse zu neuen Spalten
    df["misspelled_words"] = spellcheck_results.apply(lambda x: x["misspelled"])
    df["corrected_words"] = spellcheck_results.apply(lambda x: x["corrected"])

    return df

# Anwendung der Funktion
df_cleaned = apply_spell_to_dataframe(df_cleaned, "tweet_cleaned")

# Filtern der Zeilen mit falsch geschriebenen Wörtern
df_filtered = df_cleaned[df_cleaned["misspelled_words"].apply(len) > 0]

# Anzeigen der gewünschten Spalten
df_filtered[['misspelled_words', 'corrected_words', 'tweet', 'tweet_cleaned','label']]





##### Schimpfwörter

In [21]:
df_analyse_label_0= df_cleaned[df_cleaned['label']==0]
df_analyse_label_1= df_cleaned[df_cleaned['label']==1]
# Anzahl der Tweets mit "**" (gesamt und pro Label)
anzahl_gesamt = df_cleaned['tweet'].str.contains(r'\*\*', na=False).sum()
anzahl_label_1 = df_cleaned[df_cleaned['label'] == 1]['tweet'].str.contains(r'\*\*', na=False).sum()

# Anteil berechnen
anteil = (anzahl_label_1 / anzahl_gesamt) * 100 if anzahl_gesamt > 0 else 0

# Ergebnisse anzeigen
print(f"Gesamtanzahl Tweets mit '**': {anzahl_gesamt}")
print(f"Anzahl in Label 1: {anzahl_label_1}")
print(f"Anteil von Label 1: {anteil:.2f}%")



Gesamtanzahl Tweets mit '**': 18
Anzahl in Label 1: 4
Anteil von Label 1: 22.22%


In [30]:
english_swear_words = [
    "asshole", "bastard", "bitch", "bollocks", "bugger", "bullshit",
    "crap", "damn", "dick", "dickhead", "faggot", "fuck", "fucked",
    "fucker", "fucking", "goddamn", "hell", "jackass", "jerk",
    "motherfucker", "piss", "prick", "shit", "shithead", "slut",
    "son of a bitch", "twat", "wanker", "whore",'offensive', 'fool'
]


In [31]:
from better_profanity import profanity

# Initialisiere die Schimpfwortliste
profanity.load_censor_words(english_swear_words)  # Deine benutzerdefinierte Liste

# Prüfen auf Schimpfwörter
df_analyse_label_1['contains_bad_words'] = df_analyse_label_1['tweet'].apply(lambda x: profanity.contains_profanity(x))

df_analyse_label_1[df_analyse_label_1['contains_bad_words']==True]

C:\Users\Nasiba\AppData\Local\Temp\ipykernel_16412\2861876478.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analyse_label_1['contains_bad_words'] = df_analyse_label_1['tweet'].apply(lambda x: profanity.contains_profanity(x))


,id,label,tweet,tweet_cleaned,user_handle,hashtags,emojis,contains_bad_words
723,750,1,everytime i wear soccer shis joie fries me and says i look mexican as fuck 😒,everytime wear soccer shis joie fry say look mexican fuck unamused,0,[],:unamused_face:,True
1597,1673,1,"bullshit, he was thrown out for being a muslim",bullshit throw muslim,0,[],NaN,True
1660,1743,1,i wanna be fucked ! my id 13479 meet me here,wanna fuck meet,0,[],NaN,True
1836,1935,1,names for women fucking hard teen,name woman fuck hard teen,0,[],NaN,True
1844,1943,1,chick gets fucked nude superhero,chick get fuck nude superhero,0,[],NaN,True
...,...,...,...,...,...,...,...,...
25532,29399,1,yeah man fuck feminism,yeah man fuck feminism,0,[],NaN,True
27228,31444,1,@user is a &amp; a sick fuck! he looks like his pedophile friend @user,amp sick fuck look like friend,2,[],NaN,True
27298,31537,1,the fuck done with #mansplaining and other bullshit.,fuck bullshit,0,['#mansplaining'],NaN,True
27355,31604,1,girl get fuck bondage leather girl,girl get fuck leather girl,0,[],NaN,True


In [33]:
# Prüfen auf Schimpfwörter
df_analyse_label_0['contains_bad_words'] = df_analyse_label_0['tweet'].apply(lambda x: profanity.contains_profanity(x))

df_analyse_label_0[df_analyse_label_0['contains_bad_words']==True]

C:\Users\Nasiba\AppData\Local\Temp\ipykernel_16412\3803445381.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analyse_label_0['contains_bad_words'] = df_analyse_label_0['tweet'].apply(lambda x: profanity.contains_profanity(x))


,id,label,tweet,tweet_cleaned,user_handle,hashtags,emojis,contains_bad_words
152,156,0,"hbd to this dick suckin tequila lovin slut, i wouldnt want u any other way 💕😘 #justalillate #butstillontime",happy dick tequila lovin slut would want way two blow kiss,0,"['#justalillate', '#butstillontime']",":two_hearts:,:face_blowing_a_kiss:",True
204,210,0,this really takes the piss. i'm so angry. just goes to show who is valued and who isn't. you complete and utter moron.,really take piss angry go show value complete utter moron,0,[],NaN,True
281,289,0,if you have never lost a loved one to senseless violence you don't get a fucking opinion on the 2nd amendment,never lose one senseless violence get fucking opinion nd amendment,0,[],NaN,True
288,296,0,twinklatinboys - na: #slut #snapshot #hot #nasty #naughty #sexy #horny #shy #porn #nude #kinky #xxx #y...,na slut snapshot hot nasty naughty sexy horny shy porn nude kinky xxx,0,"['#slut', '#snapshot', '#hot', '#nasty', '#naughty', '#sexy', '#horny', '#shy', '#porn', '#nude', '#kinky', '#xxx', '#y']",NaN,True
317,325,0,sexy as fuck #directioner #niall #nialhoranfacts #niallerwins #hot #justindrewbieber #justindb #believetou...,sexy fuck hot,0,"['#directioner', '#niall', '#nialhoranfacts', '#niallerwins', '#hot', '#justindrewbieber', '#justindb', '#believetou']",NaN,True
...,...,...,...,...,...,...,...,...
27473,31742,0,i dont want a world like this... homophonia? its not a phobie there are just asshole! #praygay #orlando #prayfororlando,want world like asshole orlando prayfororlando,0,"['#praygay', '#orlando', '#prayfororlando']",NaN,True
27527,31809,0,so much shit going on in the world... ☹️. 50 people. geeesh!,much shit go world frown people,0,[],:frowning_face:,True
27561,31854,0,@user shit i just realized your account isn't a month old. it's only a few weeks old. that's more like 110 tweets a day. just,shit realize account month old week old like tweet,1,[],NaN,True
27605,31909,0,"if dudes wana fuck eachother n get married, i can never be mad at tht. jus means more females for me! #damn50 #orlando # #omg50 #gay",dude fuck n get marry never mad tht jus mean female damn orlando omg gay,0,"['#damn50', '#orlando', '#omg50', '#gay']",NaN,True
